In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
import keras_tuner
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

%matplotlib inline

In [2]:
def r2(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res / (SS_tot + K.epsilon())

In [6]:
data = pd.read_parquet('../data/features_x1.parquet')
data.head()

X = data.drop(columns=['y1', 'y2'])
y = data[['y1', 'y2']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [35]:
def build_model(hp):

    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(
            units=hp.Int("units", min_value=8, max_value=256, step=8),
            activation=hp.Choice("activation", ["relu", "leaky_relu"]),
        )
    )

    if hp.Boolean("2Layers"):
        model.add(
            tf.keras.layers.Dense(
                units=hp.Int(
                    "units",
                    min_value=8,
                    max_value=256,
                    step=8,
                ),
                activation=hp.Choice(
                    "activation",
                    ["relu", "leaky_relu"],
                ),
            )
        )

    model.add(
        tf.keras.layers.Dense(2, activation="linear"),
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[r2],
    )

    return model

In [36]:
tuner = keras_tuner.BayesianOptimization(
    build_model, objective="val_loss", num_initial_points=10, max_trials=30, seed=123, overwrite=True, project_name="ic2_tuner"
)

In [37]:
tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Trial 30 Complete [00h 01m 19s]
val_loss: 0.20120757818222046

Best val_loss So Far: 0.1936948597431183
Total elapsed time: 00h 34m 47s
INFO:tensorflow:Oracle triggered exit


In [41]:
tuner.results_summary(num_trials=1)

Results summary
Results in .\ic2_tuner
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 22 summary
Hyperparameters:
units: 176
activation: relu
2Layers: True
Score: 0.1936948597431183


In [42]:
hp = keras_tuner.HyperParameters()
hp.Fixed("activation", value="relu")
hp.Fixed("2Layers", value=True)
hp.Int("units", min_value=160, max_value=192, step=2)

tuner_rnd = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=10,
    hyperparameters=hp,
    project_name="ic2_tuner_rnd"
)

In [43]:
tuner_rnd.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Trial 10 Complete [00h 01m 16s]
val_loss: 0.21492420136928558

Best val_loss So Far: 0.19171732664108276
Total elapsed time: 00h 13m 37s
INFO:tensorflow:Oracle triggered exit


In [44]:
tuner_rnd.results_summary(num_trials=1)

Results summary
Results in .\ic2_tuner_rnd
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 04 summary
Hyperparameters:
activation: relu
2Layers: True
units: 190
Score: 0.19171732664108276
